In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('alyien_news_data.csv')
df=df.drop(['time', 'description', 'url', 'summary'], axis=1)
df.head()

,date,source,title,type,content
0,2019-08-03,Hindustan Times,"UP man ‘loses’ wife in gambling, lets friends ...",rape,"In a horrifying incident, a man addicted to ga..."
1,2019-08-03,Hindustan Times,Cong launches signature campaign for justice t...,rape,Lucknow Uttar Pradesh Congress on Saturday lau...
2,2019-08-03,Times of India,"Unnao rape victim develops pneumonia, remains ...",rape,"LUCKNOW: Therape victim, who was injured in a ..."
3,2019-08-04,Times of India,Jamshedpur rape case: Richa Chadha expresses h...,rape,"A three-year-old girl was kidnapped, raped and..."
4,2019-08-04,Times of India,Unnao rape survivor stable: Doctors | India Ne...,rape,LUCKNOW: The condition of the Unnao rape survi...


In [3]:
import nltk
from nltk.tag.stanford import StanfordNERTagger

jar = './stanford-ner.jar'
model = './english.all.3class.distsim.crf.ser.gz'

def findLocations(sentence):
    locations = []
    ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')
    words = nltk.word_tokenize(sentence)
    lst = ner_tagger.tag(words)
    
    i=0
    while i<len(lst):
        if lst[i][1]=='LOCATION':
            s = lst[i][0]
            i+=1
            if i==len(lst):
                break
            while lst[i][1]=='LOCATION':
                s = s+' '+lst[i][0]
                i+=1
                if i==len(lst):
                    break
            if s.lower() not in locations:
                locations.append(s.lower())
        else:
            i+=1
        
    return locations    

In [5]:
contents = ' '.join(list(df['content']))
locations = findLocations(contents)

In [6]:
f=open('loc.txt', 'r')
s=f.read()

In [7]:
loc = s.split('\n')[3:]
delhi_locs = []
for i in loc:
    if i!='LOCALITY':
        delhi_locs.append(i.lower())

In [8]:
all_locations = list(set(delhi_locs).intersection(locations))
len(all_locations)

52

In [9]:
all_locations = sorted(all_locations)

In [18]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myGeocoder")

In [28]:
ignore_loc = []
for i in range(len(all_locations)):
    loc = all_locations[i]
    flag = 0
    while flag==0:
        try:
            location = geolocator.geocode(loc+" delhi", timeout=10)
            if location is None:
                ignore_loc.append(loc)
                print(loc)
            flag = 1
        except Exception as e:
            pass  

gulabi bagh
keshavpuram


In [3]:
news_data = pd.read_csv('news_data_2.csv')
news_data.head()

,location,date,headline,crime_type,article,age,businessman,percentile
0,civil lines,2019-10-31\t2019-11-01\t2019-09-05,UP: Karwariya brothers convicted for SP MLA's ...,murder\tmurder\tmurder,"ALLAHABAD: Former BSP MP Kapil Muni Karwariya,...",-1\t-1\t-1,0\t0\t1,50
1,mehrauli,2019-11-01\t2019-11-01\t2019-11-02\t2019-09-18...,HC notice to Delhi govt on Jessica Lal murder ...,murder\tmurder\tmurder\tmurder\tmurder,The Delhi High Court on Friday issued a notice...,-1\t-1\t-1\t-1\t-1,0\t0\t0\t0\t0,17
2,tis hazari,2019-08-05\t2019-08-05,15 cops in Unnao rape survivor’s security quiz...,rape\trape,"Fifteen police constables, including nine of n...",-1\t-1,0\t0,67
3,fatehpur beri,2019-08-07,"On bail in rape case, man robs milk trader of ...",rape,New Delhi An man accused of raping a minor gir...,21,0,84
4,aiims,2019-08-09\t2019-09-25,Unnao case: Court files charges of rape agains...,rape\trape,A Delhi court on Friday framed charges against...,-1\t-1,0\t0,67


In [42]:
crime_count = {}
for i in range(len(news_data)):
    loc = news_data['location'].iloc[i]
    crime_count[loc] = len(list(str(news_data['date'].iloc[i]).split('\t')))
print(crime_count)

{'civil lines': 3, 'mehrauli': 5, 'tis hazari': 2, 'fatehpur beri': 1, 'aiims': 2, 'chhawla': 2, 'okhla': 4, 'yamuna vihar': 1, 'kalkaji': 2, 'palam': 1, 'khanpur': 1, 'bawana': 1, 'sangam vihar': 2, 'kashmiri gate': 4, 'geeta colony': 1, 'rohini': 4, 'kirti nagar': 1, 'tughlakabad': 1, 'shalimar bagh': 2, 'narela': 5, 'wazirabad': 2, 'burari': 1, 'chittaranjan park': 1, 'dwarka': 6, 'kakrola': 1, 'najafgarh': 2, 'prem nagar': 1, 'saket': 1, 'pitampura': 2, 'lajpat nagar': 1, 'gulabi bagh': 2, 'karawal nagar': 1, 'kapashera': 3, 'naraina': 1, 'rajouri garden': 1, 'tilak nagar': 1, 'mundka': 1, 'nangloi': 1, 'connaught place': 1, 'mangolpuri': 1, 'inderpuri': 1, 'green park': 1, 'jasola': 2, 'vasant kunj': 2, 'vijay nagar': 1, 'sarai kale khan': 1, 'chhatarpur': 2, 'keshavpuram': 1, 'trilokpuri': 2}


In [34]:
rows = [['source', 'destination', 'distance', 'source_crime_count', 'destination_crime_count']]

In [35]:
from haversine import haversine, Unit
for i in range(len(all_locations)):
    if all_locations[i] in ignore_loc:
        continue
        
    flag_1 = 0
    cord_1 = []
    while flag_1==0:
        try:
            location = geolocator.geocode(all_locations[i]+" delhi", timeout=10)
            cord_1.append(location.latitude)
            cord_1.append(location.longitude)
            flag_1 = 1
        except Exception:
            pass
        
    for j in range(i+1, len(all_locations)):
        if all_locations[j] in ignore_loc:
            continue
            
        flag_2 = 0
        cord_2 = []
        while flag_2==0:
            try:
                location = geolocator.geocode(all_locations[j]+" delhi", timeout=10)
                cord_2.append(location.latitude)
                cord_2.append(location.longitude)
                flag_2 = 1
            except Exception:
                pass
        dist = haversine(cord_1, cord_2)
        print(all_locations[i], '\t', all_locations[j], '\t', dist)
        
        row = [all_locations[i], all_locations[j], dist]
        if all_locations[i] in crime_count.keys():
            row.append(crime_count[all_locations[i]])
        else:
            row.append(0)
            
        if all_locations[j] in crime_count.keys():
            row.append(crime_count[all_locations[j]])
        else:
            row.append(0)
            
        rows.append(row)

adarsh nagar 	 aiims 	 14.299678642456529
adarsh nagar 	 bawana 	 20.964843913882373
adarsh nagar 	 burari 	 19.64475654376608
adarsh nagar 	 chhatarpur 	 15.643106849873972
adarsh nagar 	 chhawla 	 8.701591032985172
adarsh nagar 	 chittaranjan park 	 19.274258148635532
adarsh nagar 	 civil lines 	 16.522029593571492
adarsh nagar 	 connaught place 	 14.595941016411253
adarsh nagar 	 dwarka 	 4.759881728964022
adarsh nagar 	 fatehpur beri 	 20.533329235383036
adarsh nagar 	 gautam nagar 	 14.927123384356833
adarsh nagar 	 geeta colony 	 20.34265665064637
adarsh nagar 	 green park 	 14.611987041881196
adarsh nagar 	 inderpuri 	 7.548889390274024
adarsh nagar 	 jasola 	 23.184745796279334
adarsh nagar 	 kakrola 	 5.3922219593514615
adarsh nagar 	 kalkaji 	 20.462901708082544
adarsh nagar 	 kapashera 	 9.719578731675067
adarsh nagar 	 karawal nagar 	 23.71265709105325
adarsh nagar 	 kashmiri gate 	 16.84804966051284
adarsh nagar 	 khanpur 	 19.336328225789654
adarsh nagar 	 kirti nagar 	 8

In [56]:
import csv
loc_distance = open('location_distance.csv', 'w')
writer = csv.writer(loc_distance)
writer.writerows(rows)
loc_distance.close()

In [ ]:
df